### The grand quest: make it actually work (4 points)

Your main task is to use some of the tricks you've learned on the network and analyze if you can improve __validation MAE__. Try __at least 3 options__ from the list below for a passing grade. Write a short report about what you have tried. More ideas = more bonus points. 

__Please be serious:__ " plot learning curves in MAE/epoch, compare models based on optimal performance, test one change at a time. You know the drill :)

You can use either pure __tensorflow__ or __keras__. Feel free to adapt the seminar code for your needs.


In [1]:
import os

import pandas as pd
import numpy as np
import nltk
from collections import Counter
import gensim.downloader 
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
# import tensorflow.contrib.keras as keras
import keras
import tensorflow as tf

import keras.layers as L
from tensorflow.contrib.keras import activations

from keras.backend.tensorflow_backend import set_session

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
def get_session(gpu_fraction=.8):
    num_threads = os.environ.get('OMP_NUM_THREADS')
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)
    if num_threads:
        return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, intra_op_parallelism_threads=num_threads))
    else:
        return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [3]:
set_session(get_session(.8))

In [4]:
data = pd.read_csv("./Train_rev1.csv", index_col=None)
data.shape

(244768, 12)

In [5]:
data['Log1pSalary'] = np.log1p(data['SalaryNormalized'])

In [6]:
text_columns = ["Title", "FullDescription"]
categorical_columns = ["Category", "Company", "LocationNormalized", "ContractType", "ContractTime"]
target_column = "Log1pSalary"

data[categorical_columns] = data[categorical_columns].fillna('NaN') # cast missing values to string "NaN"

In [7]:
tokenizer = nltk.tokenize.WordPunctTokenizer()

In [8]:
data[text_columns] = data[text_columns].fillna(' ')

In [9]:
# see task above
# ' '.join(tokenizer.tokenize('dsfds dsdsg, 435g f'.casefold()))
data['Title'] = data['Title'].apply(lambda s: ' '.join(tokenizer.tokenize(s.casefold())))
data['FullDescription'] = data['FullDescription'].apply(lambda s: ' '.join(tokenizer.tokenize(s.casefold())))

In [10]:
# token_counts = Counter()

In [11]:
# _ = data["FullDescription"].apply(lambda s: token_counts.update(s.split()))
# _ = data["Title"].apply(lambda s: token_counts.update(s.split()))

In [12]:
# min_count = 10

# tokens from token_counts keys that had at least min_count occurrences throughout the dataset
# tokens = [key for key, count in token_counts.items() if count>=10]

In [13]:
UNK, PAD = "UNK", "PAD"
# tokens = [UNK, PAD] + sorted(tokens)

In [14]:
embeddings = gensim.downloader.load("fasttext-wiki-news-subwords-300")
# embeddings = gensim.downloader.load("glove-wiki-gigaword-100")

In [15]:
# tokens = [key for key in embeddings.vocab.keys()]

In [16]:
# token_to_id = {token: tokens.index(token) for token in tokens}

In [17]:
# UNK_IX, PAD_IX = map(token_to_id.get, [UNK, PAD])

In [18]:
UNK_IX, PAD_IX = map(embeddings.vocab.get, [UNK, PAD])
UNK_IX, PAD_IX = UNK_IX.index, PAD_IX.index

In [19]:
def as_matrix(sequences, max_len=None):
    """ Convert a list of tokens into a matrix with padding """
    if isinstance(sequences[0], str):
        sequences = list(map(str.split, sequences))
        
    max_len = min(max(map(len, sequences)), max_len or float('inf'))
    
    matrix = np.full((len(sequences), max_len), np.int32(PAD_IX))
    for i,seq in enumerate(sequences):
        row_ix = [embeddings.vocab.get(word, embeddings.vocab.get('UNK')).index for word in seq[:max_len]]
#         row_ix = [token_to_id.get(word, UNK_IX) for word in seq[:max_len]]
        matrix[i, :len(row_ix)] = row_ix
    
    return matrix

In [20]:
# we only consider top-1k most frequent companies to minimize memory usage
top_companies, top_counts = zip(*Counter(data['Company']).most_common(1000))
recognized_companies = set(top_companies)
data["Company"] = data["Company"].apply(lambda comp: comp if comp in recognized_companies else "Other")

categorical_vectorizer = DictVectorizer(dtype=np.float32, sparse=False)
categorical_vectorizer.fit(data[categorical_columns].apply(dict, axis=1))

DictVectorizer(dtype=<class 'numpy.float32'>, separator='=', sort=True,
        sparse=False)

In [21]:
data_train, data_val = train_test_split(data, test_size=0.2, random_state=42)
data_train.index = range(len(data_train))
data_val.index = range(len(data_val))

print("Train size = ", len(data_train))
print("Validation size = ", len(data_val))

Train size =  195814
Validation size =  48954


In [22]:
def make_batch(data, max_len=None, word_dropout=0):
    """
    Creates a keras-friendly dict from the batch data.
    :param word_dropout: replaces token index with UNK_IX with this probability
    :returns: a dict with {'title' : int64[batch, title_max_len]
    """
    batch = {}
    batch["Title"] = as_matrix(data["Title"].values, max_len)
    batch["FullDescription"] = as_matrix(data["FullDescription"].values, max_len)
    batch['Categorical'] = categorical_vectorizer.transform(data[categorical_columns].apply(dict, axis=1))
    
    if word_dropout != 0:
        batch["FullDescription"] = apply_word_dropout(batch["FullDescription"], 1. - word_dropout)
    
    if target_column in data.columns:
        batch[target_column] = data[target_column].values
    
    return batch

def apply_word_dropout(matrix, keep_prop, replace_with=UNK_IX, pad_ix=PAD_IX,):
    dropout_mask = np.random.choice(2, np.shape(matrix), p=[keep_prop, 1 - keep_prop])
    dropout_mask &= matrix != pad_ix
    return np.choose(dropout_mask, [matrix, np.full_like(matrix, replace_with)])

In [23]:
def build_model(n_tokens=len(embeddings.vocab), n_cat_features=len(categorical_vectorizer.vocabulary_), hid_size=64):
    """ Build a model that maps three data sources to a single linear output: predicted log1p(salary) """
    
    l_title = L.Input(shape=[None], name="Title")
    l_descr = L.Input(shape=[None], name="FullDescription")
    l_categ = L.Input(shape=[n_cat_features], name="Categorical")
    
    # Build your monster!
    Emb_layer = embeddings.get_keras_embedding()
#     Emb_layer = L.Embedding(n_tokens, 40)
    Ttl_conv = L.Conv1D(32, 2, activation=activations.tanh)
    Dsc_conv = L.Conv1D(32, 3, activation=activations.tanh)
    Gpool = L.GlobalMaxPool1D()
    
    ttl_emb = Emb_layer(l_title)
    t_conv = Ttl_conv(ttl_emb)
    t_pool = Gpool(t_conv)
    
    dsc_emb = Emb_layer(l_descr)
    dsc_conv = Dsc_conv(dsc_emb)
    dsc_pool = Gpool(dsc_conv)
    
    cat_repr = L.Dense(100, activation=activations.tanh)(l_categ)
    
    concat = L.Concatenate()([t_pool, dsc_pool, cat_repr])
    densed = L.Dense(100, activation=activations.tanh)(concat)
    output_layer = L.Dense(1, activation=activations.relu)(densed)
    # end of your code
    
    model = keras.models.Model(inputs=[l_title, l_descr, l_categ], outputs=[output_layer])
    model.compile('adam', 'mean_squared_error', metrics=['mean_absolute_error'])
    return model
    

In [24]:
model = build_model()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Title (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
FullDescription (InputLayer)    (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    299999700   Title[0][0]                      
                                                                 FullDescription[0][0]            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, None, 32)     19232       embedding_1[0][0]                
__________

In [25]:
def iterate_minibatches(data, batch_size=256, shuffle=True, cycle=False, **kwargs):
    """ iterates minibatches of data in random order """
    while True:
        indices = np.arange(len(data))
        if shuffle:
            indices = np.random.permutation(indices)

        for start in range(0, len(indices), batch_size):
            batch = make_batch(data.iloc[indices[start : start + batch_size]], **kwargs)
            target = batch.pop(target_column)
            yield batch, target
        
        if not cycle: break

In [ ]:
batch_size = 256
epochs = 20            # definitely too small
steps_per_epoch = 100  # for full pass over data: (len(data_train) - 1) // batch_size + 1

model = build_model()

model.fit_generator(iterate_minibatches(data_train, batch_size, cycle=True, word_dropout=0.05), 
                    epochs=epochs, steps_per_epoch=steps_per_epoch,
                    
                    validation_data=iterate_minibatches(data_val, batch_size, cycle=True),
                    validation_steps=data_val.shape[0] // batch_size
                   )

Epoch 1/20
100/100 [==============================] - 19s 194ms/step - loss: 10.5736 - mean_absolute_error: 1.8066 - val_loss: 0.2118 - val_mean_absolute_error: 0.3680
Epoch 2/20
100/100 [==============================] - 17s 175ms/step - loss: 0.1962 - mean_absolute_error: 0.3507 - val_loss: 0.1797 - val_mean_absolute_error: 0.3339
Epoch 3/20
100/100 [==============================] - 18s 177ms/step - loss: 0.1710 - mean_absolute_error: 0.3247 - val_loss: 0.1622 - val_mean_absolute_error: 0.3154
Epoch 4/20
100/100 [==============================] - 18s 181ms/step - loss: 0.1585 - mean_absolute_error: 0.3109 - val_loss: 0.1531 - val_mean_absolute_error: 0.3055
Epoch 5/20
100/100 [==============================] - 18s 183ms/step - loss: 0.1493 - mean_absolute_error: 0.3001 - val_loss: 0.1477 - val_mean_absolute_error: 0.2985
Epoch 6/20
100/100 [==============================] - 18s 179ms/step - loss: 0.1437 - mean_absolute_error: 0.2939 - val_loss: 0.1387 - val_mean_absolute_error: 0.28

In [27]:
def print_metrics(model, data, batch_size=batch_size, name="", **kw):
    squared_error = abs_error = num_samples = 0.0
    for batch_x, batch_y in iterate_minibatches(data, batch_size=batch_size, shuffle=False, **kw):
        batch_pred = model.predict(batch_x)[:, 0]
        squared_error += np.sum(np.square(batch_pred - batch_y))
        abs_error += np.sum(np.abs(batch_pred - batch_y))
        num_samples += len(batch_y)
    print("%s results:" % (name or ""))
    print("Mean square error: %.5f" % (squared_error / num_samples))
    print("Mean absolute error: %.5f" % (abs_error / num_samples))
    return squared_error, abs_error
    
print_metrics(model, data_train, name='Train')
print_metrics(model, data_val, name='Val');

Train results:
Mean square error: 0.08662
Mean absolute error: 0.21968
Val results:
Mean square error: 0.08769
Mean absolute error: 0.22234


In [12]:
def func(total, rate, monthly, length):
    print(1+rate/100/12)
    for i in range(length):
        total -= monthly
        total *= 1+rate/100/12
    print(total)

In [53]:
func(170000, 19, 6200, 36)

1.0158333333333334
-3200.5121937785198


### A short report

Please tell us what you did and how did it work.

`<YOUR_TEXT_HERE>`, i guess...

## Recommended options

#### A) CNN architecture

All the tricks you know about dense and convolutional neural networks apply here as well.
* Dropout. Nuff said.
* Batch Norm. This time it's `L.BatchNormalization`
* Parallel convolution layers. The idea is that you apply several nn.Conv1d to the same embeddings and concatenate output channels.
* More layers, more neurons, ya know...


#### B) Play with pooling

There's more than one way to perform pooling:
* Max over time - our `L.GlobalMaxPool1D`
* Average over time (excluding PAD)
* Softmax-pooling:
$$ out_{i, t} = \sum_t {h_{i,t} \cdot {{e ^ {h_{i, t}}} \over \sum_\tau e ^ {h_{j, \tau}} } }$$

* Attentive pooling
$$ out_{i, t} = \sum_t {h_{i,t} \cdot Attn(h_t)}$$

, where $$ Attn(h_t) = {{e ^ {NN_{attn}(h_t)}} \over \sum_\tau e ^ {NN_{attn}(h_\tau)}}  $$
and $NN_{attn}$ is a dense layer.

The optimal score is usually achieved by concatenating several different poolings, including several attentive pooling with different $NN_{attn}$ (aka multi-headed attention).

The catch is that keras layers do not inlude those toys. You will have to [write your own keras layer](https://keras.io/layers/writing-your-own-keras-layers/). Or use pure tensorflow, it might even be easier :)

#### C) Fun with words

It's not always a good idea to train embeddings from scratch. Here's a few tricks:

* Use a pre-trained embeddings from `gensim.downloader.load`. See last lecture.
* Start with pre-trained embeddings, then fine-tune them with gradient descent. You may or may not want to use __`.get_keras_embedding()`__ method for word2vec
* Use the same embedding matrix in title and desc vectorizer


#### D) Going recurrent

We've already learned that recurrent networks can do cool stuff in sequence modelling. Turns out, they're not useless for classification as well. With some tricks of course..

* Like convolutional layers, LSTM should be pooled into a fixed-size vector with some of the poolings.
* Since you know all the text in advance, use bidirectional RNN
  * Run one LSTM from left to right
  * Run another in parallel from right to left 
  * Concatenate their output sequences along unit axis (dim=-1)

* It might be good idea to mix convolutions and recurrent layers differently for title and description


#### E) Optimizing seriously

* You don't necessarily need 100 epochs. Use early stopping. If you've never done this before, take a look at [early stopping callback](https://keras.io/callbacks/#earlystopping).
  * In short, train until you notice that validation
  * Maintain the best-on-validation snapshot via `model.save(file_name)`
  * Plotting learning curves is usually a good idea
  
Good luck! And may the force be with you!